In [3]:
#!/usr/bin/env python3
"""
Simulación 3D simple de un cohete de agua (botella 1 L, 1/3 agua, 3 bar gauge),
y creación de una animación GIF con matplotlib.

Guarda: trajectory3d.csv y trajectory3d.gif en el directorio de ejecución.
"""

import numpy as np
import pandas as pd
from math import sqrt, cos, sin
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter

# ---------- Parámetros físicos ----------
g = 9.80665
rho_air = 1.225
rho_water = 997.0
gamma = 1.4
Pa = 101325.0

mass_dry = 0.15       # kg
mass_water0 = 0.33    # kg (1/3 L)
P0 = 4.0e5            # Pa (4 bar absolutos)
V_bottle = 1.0e-3     # m^3 (1 L)
An = 5.0e-5           # m^2 (8 mm diámetro)
Cd = 0.8
Cd_drag = 0.5
A_front = 0.005       # m^2 (área frontal aproximada)
launch_angle_deg = 60.0
azimuth_deg = 0.0

# ---------- Simulación (resolución que puedes ajustar) ----------
dt = 0.005      # s (aumenta para acelerar, reduce para más precisión)
tmax = 5.0      # s (tiempo máximo)
theta = np.deg2rad(launch_angle_deg)
azi = np.deg2rad(azimuth_deg)

# pequeña perturbación en azimuth para generar 3D
rng = np.random.default_rng(123)
azi += rng.uniform(-0.12, 0.12)

# estado inicial
t = 0.0
x = 0.0; y = 0.0; z = 0.0
vx = vy = vz = 0.0

mass_water = mass_water0
mass_total = mass_dry + mass_water
V_air0 = V_bottle - mass_water0 / rho_water
const_PV = P0 * (V_air0**gamma)
P = P0

times, Xs, Ys, Zs = [], [], [], []
Vxs, Vys, Vzs = [], [], []
Masses, Ps = [], []

nsteps = int(tmax/dt)
for i in range(nsteps+1):
    times.append(t); Xs.append(x); Ys.append(y); Zs.append(z)
    Vxs.append(vx); Vys.append(vy); Vzs.append(vz)
    Masses.append(mass_total); Ps.append(P)

    # parar si ya aterrizó
    if y <= 0 and t > 0.05:
        break

    # empuje
    thrust = 0.0; mdot = 0.0
    if mass_water > 1e-8 and P > Pa:
        V_air = V_bottle - mass_water / rho_water
        Ve = Cd * np.sqrt(max(0.0, 2.0*(P-Pa)/rho_water))
        mdot = rho_water * An * Ve
        if mdot*dt > mass_water:
            mdot = mass_water / dt
            Ve = mdot / (rho_water*An) if An>0 else 0.0
        thrust = mdot * Ve

    # jitter para 3D visual
    jitter = 0.02 * np.sin(8.0*t)

    # arrastre
    v = sqrt(vx*vx + vy*vy + vz*vz)
    if v > 1e-12:
        drag = 0.5 * rho_air * Cd_drag * A_front * v*v
        Fx = -drag * (vx / v); Fy = -drag * (vy / v); Fz = -drag * (vz / v)
    else:
        Fx = Fy = Fz = 0.0

    ux = cos(theta)*cos(azi)
    uy = sin(theta)
    uz = cos(theta)*sin(azi)
    ux = ux + jitter * 0.08
    uz = uz - jitter * 0.03
    norm_u = sqrt(ux*ux + uy*uy + uz*uz)
    ux /= norm_u; uy /= norm_u; uz /= norm_u

    ax = (thrust*ux + Fx) / mass_total
    ay = (thrust*uy + Fy) / mass_total - g
    az_acc = (thrust*uz + Fz) / mass_total

    # integración RK4 simplificada (masa cambia poco en dt)
    k1vx = ax; k1vy = ay; k1vz = az_acc
    k1x = vx; k1y = vy; k1z = vz

    vx2 = vx + 0.5*dt*k1vx; vy2 = vy + 0.5*dt*k1vy; vz2 = vz + 0.5*dt*k1vz
    k2vx = ax; k2vy = ay; k2vz = az_acc
    k2x = vx2; k2y = vy2; k2z = vz2

    vx3 = vx + 0.5*dt*k2vx; vy3 = vy + 0.5*dt*k2vy; vz3 = vz + 0.5*dt*k2vz
    k3vx = ax; k3vy = ay; k3vz = az_acc
    k3x = vx3; k3y = vy3; k3z = vz3

    vx4 = vx + dt*k3vx; vy4 = vy + dt*k3vy; vz4 = vz + dt*k3vz
    k4vx = ax; k4vy = ay; k4vz = az_acc
    k4x = vx4; k4y = vy4; k4z = vz4

    x += dt*(k1x + 2*k2x + 2*k3x + k4x)/6.0
    y += dt*(k1y + 2*k2y + 2*k3y + k4y)/6.0
    z += dt*(k1z + 2*k2z + 2*k3z + k4z)/6.0
    vx += dt*(k1vx + 2*k2vx + 2*k3vx + k4vx)/6.0
    vy += dt*(k1vy + 2*k2vy + 2*k3vy + k4vy)/6.0
    vz += dt*(k1vz + 2*k2vz + 2*k3vz + k4vz)/6.0

    # actualizar masa y presión
    mass_water = mass_water - mdot*dt
    if mass_water < 0.0: mass_water = 0.0
    mass_total = mass_dry + mass_water
    V_air = V_bottle - mass_water / rho_water
    if V_air < 1e-12: V_air = 1e-12
    P = const_PV / (V_air**gamma)

    t += dt

# Guardar CSV
df = pd.DataFrame({
    't': times, 'x': Xs, 'y': Ys, 'z': Zs,
    'vx': Vxs, 'vy': Vys, 'vz': Vzs,
    'mass': Masses, 'P': Ps
})
df.to_csv('trajectory3d.csv', index=False)
print("CSV guardado: trajectory3d.csv")

# ---------- Crear animación 3D y guardar GIF ----------
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_zlabel('z (m)')
ax.set_title('Trayectoria 3D - Cohete de Agua')

ax.set_xlim(0, max(1.0, max(df['x'])+0.5))
ax.set_ylim(0, max(1.0, max(df['y'])+0.5))
zlim = max(abs(min(df['z'])), abs(max(df['z'])))+0.5
ax.set_zlim(-zlim, zlim)

line, = ax.plot([], [], [], lw=2)
point, = ax.plot([], [], [], marker='o')

N = len(df)
def init():
    line.set_data([], []); line.set_3d_properties([])
    point.set_data([], []); point.set_3d_properties([])
    return line, point

def update(frame):
    i = frame
    xdata = df['x'].values[:i]
    ydata = df['y'].values[:i]
    zdata = df['z'].values[:i]
    line.set_data(xdata, ydata); line.set_3d_properties(zdata)
    point.set_data(df['x'].values[i-1:i], df['y'].values[i-1:i])
    point.set_3d_properties(df['z'].values[i-1:i])
    ax.view_init(elev=20, azim=20 + 240.0*(i/N))
    return line, point

anim = animation.FuncAnimation(fig, update, frames=range(1, N), init_func=init, interval=30, blit=True)

# puedes bajar fps o dpi para un GIF más ligero
gif_name = 'trajectory3d.gif'
writer = PillowWriter(fps=15)
anim.save(gif_name, writer=writer)
plt.close(fig)
print("GIF guardado:", gif_name)



CSV guardado: trajectory3d.csv
GIF guardado: trajectory3d.gif


In [2]:
pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 20.6 MB/s  0:00:00a 0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pandas]━━━━ 1/2 [pandas]
Note: you may need to restart the kernel to use updated packages.
